author: Andre Scholich
<br>
last update: 2016-04-09


# Performance strategies

Since python is an interpreted programming language (as also e.g. Matlab) it may execute code significantly slower than compiled languages such as C or Fortran.
There are now several option of how to deal with this issue.
Here, I will follow mainly the example of [Numba vs Cython](https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/) from Jake Vanderplas.


# Make use of scientific libraries

The most straight-forward recommendation is to resort to scientific libraries, in particular numpy and scipy to perform standard tasks such as summation or multiplication of arrays.
To illustrate the significance of this point let us examine some execution times.

In [6]:
from pylab import *
arr = np.random.random(10000)

In [7]:
def sum_naive(arr):
    """Take the sum over an array using a python for loop.
    """
    ret = 0.0
    for a in arr:
        ret += a
    return ret

First the naive sum using the python for loop.

In [8]:
%timeit sum_naive(arr)

100 loops, best of 3: 1.65 ms per loop


Now the same sum using the precompiled numpy sum function.

In [9]:
%timeit np.sum(arr)

The slowest run took 6.42 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 7.16 µs per loop


We observe that the execution of the naive sum is several hundred times slower.

# Compile your own function

Sometimes one needs custom functions, which are unavailable in the current scientific packages. In that case on can use either cython, numba, f2py or other ways to compile code into this low-lever languages and use it from within python in the same way as shown for numpy above.

Let us take the pairwise distance as a test case.

In [10]:
x = np.random.random((1000, 3))

In [11]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D
%timeit pairwise_python(x)

1 loops, best of 3: 2.84 s per loop


Note that this particular example can also be solved by resorting to numpy functions and broadcasting, which has the drawback of using much more memory (M * N * M) instead of M * M entries as it creates a large temporary array.

In [12]:
def pairwise_numpy(X):
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))
%timeit pairwise_numpy(x)

10 loops, best of 3: 52 ms per loop


## Numba wrapper

In short, numba takes the function and compiles it during execution time (not pre-compiled). Hence the name "just-in-time" compiler.
Usage is apparently very simple. One calls the function `autojit` on the pure-python implementation and gets back a faster variant.

In [13]:
from numba import double
from numba.decorators import jit, autojit

pairwise_numba = autojit(pairwise_python)

%timeit pairwise_numba(x)

The slowest run took 21.16 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 10.2 ms per loop


## Cython

As numba seems like magic and one might want more control over what is going on one can also pre-compile c (or c++) code using cython.
For simplicity, an IPython magic function is used here but the same can be achieved in a similar manner in pure python code.

In [14]:
import setuptools

In [15]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [16]:
%%cython
import numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = np.empty((M, M), dtype=np.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return np.asarray(D)

In [17]:
%timeit pairwise_cython(x)

100 loops, best of 3: 9.08 ms per loop


# Parallize code

## IPython parallel

Module for easy [parallel calculations](https://ipython.org/ipython-doc/3/parallel/parallel_intro.html).
Install with conda

    conda install ipyparallel
    
and start a cluster of four processes in this case.

    ipcluster start -n 8
    
Now we create a client and a direct view object on that client. In short, this gives a handle on all the instances of the cluster and we can perform the same operation on all of them, such as importing modules or defining functions by the `%px` line magic.

In [20]:
from ipyparallel import Client
pc = Client()
dview = pc[:]

# import time on all remote instances, if you did not start the cluster or it is still starting up you will get an error
%px import time
# and also import it here in the notebook for reference
import time

In [1]:
def wait(i):
    time.sleep(.5)
    return i

In [2]:
%%time
result = map(wait, range(8))
print result

NameError: global name 'time' is not defined

In [3]:
%%time
worker = dview.map_async(wait, range(8))
result = worker.get()
print result

NameError: name 'dview' is not defined

## Without IPython using `multiprocessing`

Basic module for creating a pool of threads and running on several cpu cores in parallel.

[website](https://pymotw.com/2/multiprocessing/basics.html)

In [4]:
from multiprocessing import Pool

po = Pool(8)

In [ ]:
pool = Pool(processes=8)              # start 8 worker processes
# result = pool.apply_async(f, range(8))
# print result.get(timeout=1)           # prints "100" unless your computer is *very* slow
print pool.map(wait, range(10))          # prints "[0, 1, 4,..., 81]"